# Implementación el método de la máxima verosimilitud para la regresión logística

### Definir la función de entorno L(b)

In [1]:
from IPython.display import display, Math, Latex
display(Math(r'L(\beta)=\sum_{i=1}^n P_i^{y_i}(1-Pi)^{y_i}'))


<IPython.core.display.Math object>

In [2]:
def likelihood(y, pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1, len(y)+1))
    for i in range(len(y)):
        sum_in[i] = np.where(y[i]==1, pi[i], 1-pi[i])
        total_sum = total_sum * sum_in[i]
    return total_sum

## Calcular las probabilidades por cada observación 

In [3]:
display(Math(r'P_i = P(x_i) = \frac{1}{1+e^{-\sum_{j=0}^k\beta_j\cdot x_{ij}}} '))

<IPython.core.display.Math object>

In [4]:
def logitprobs(X,beta):
    import numpy as np
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi=list(range(1,n_rows+1))
    expon=list(range(1,n_rows+1))
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex=X[i][j] * beta[j]
            expon[i] = ex + expon[i]
        with np.errstate(divide="ignore", invalid="ignore"):
            pi[i]=1/(1+np.exp(-expon[i]))
    return pi

### Calcular la matriz diagonal W

In [5]:
display(Math(r'W= diag(P_i \cdot (1-P_i))_{i=1}^n'))

<IPython.core.display.Math object>

In [6]:
def findW(pi):
    import numpy as np
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n)
    for i in range(n):
        print(i)
        W[i,i]=pi[i]*(1-pi[i])
        W[i,i].astype(float)
    return W

### Obtener la solución de la función logística

In [7]:
display(Math(r"\beta_{n+1} = \beta_n -\frac{f(\beta_n)}{f'(\beta_n)}"))
display(Math(r"f(\beta) = X(Y-P)"))
display(Math(r"f'(\beta) = XWX^T"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [8]:
def logistics(X, Y, limit):
    import numpy as np
    from numpy import linalg
    nrow = np.shape(X)[0]
    bias = np.ones(nrow).reshape(nrow,1)
    X_new = np.append(X, bias, axis = 1)
    ncol = np.shape(X_new)[1]
    beta = np.zeros(ncol).reshape(ncol,1)
    root_dif = np.array(range(1,ncol+1)).reshape(ncol,1)
    iter_i = 10000
    while(iter_i>limit):
        print("Iter:i"+str(iter_i) + ", limit:" + str(limit))
        pi = logitprobs(X_new, beta)
        print("Pi:"+str(pi))
        W = findW(pi)
        print("W:"+str(W))
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose())
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new))
        root_dif = np.array(linalg.inv(den)*num)
        beta = beta + root_dif
        print("Beta: "+str(beta))
        iter_i = np.sum(root_dif*root_dif)
        ll = likelihood(Y, pi)
    return beta

## Comprobación experimental

In [9]:
import numpy as np

In [10]:
X = np.array(range(10)).reshape(10,1)
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [11]:
Y = [0,0,0,0,1,1,1,1,1,1]

In [12]:
bias = np.ones(10).reshape(10,1)
X_new = np.append(X, bias, axis=1)
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [13]:
a = logistics(X,Y,0.00001)

Iter:i10000, limit:1e-05
Pi:[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
W:[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
Beta: [[ 0.58181818]
 [-2.21818182]]
Iter:i5.258842975206606, limit:1e-05
Pi:[array([0.0981296]), array([0.16296047]), array([0.2583532]), array([0.38397094]), array([0.52724571]), array([0.66616913]), array([0.78120925]), array([0.86465991]), array([0.91955851]), array([0.9533

In [14]:
ll = likelihood(Y, logitprobs(X,a))
ll

array([0.])

In [15]:
Y = 94.94002253 * X -342.81805557
Y

array([[-342.81805557],
       [-247.87803304],
       [-152.93801051],
       [ -57.99798798],
       [  36.94203455],
       [ 131.88205708],
       [ 226.82207961],
       [ 321.76210214],
       [ 416.70212467],
       [ 511.6421472 ]])

## Con el paquete stasmodel de python

In [16]:
import statsmodels.api as sm

In [17]:
Y = (Y - np.min(Y))/np.ptp(Y)
logit_model = sm.Logit(Y,X_new)

In [18]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.359693
         Iterations 6


In [19]:
print(result.summary2())

                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.481    
Dependent Variable: y                AIC:              11.1939  
Date:               2021-07-18 09:08 BIC:              11.7990  
No. Observations:   10               Log-Likelihood:   -3.5969  
Df Model:           1                LL-Null:          -6.9315  
Df Residuals:       8                LLR p-value:      0.0098099
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     6.0000                                      
------------------------------------------------------------------
           Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
------------------------------------------------------------------
x1         0.6272     0.3735    1.6793   0.0931   -0.1048   1.3592
const     -2.8224     1.8730   -1.5069   0.1318   -6.4934   0.8485

